In [1]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.utils as utils
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import ta 
import datetime

In [ ]:

csvFileAddress = input('csv file address') 

DJ = pd.read_csv(csvFileAddress,delim_whitespace=True)

DJ['<ISGREEN>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']
DJ['<SIZE>'] =  (DJ['<CLOSE>'] - DJ['<OPEN>']) / DJ['<CLOSE>'] *100
DJ['<VOLATILITY>'] =  DJ['<HIGH>'] - DJ['<LOW>'] /DJ['<CLOSE>']

DJ.drop(['<VOL>'],axis=1,inplace=True)

DJ.fillna(0,inplace=True)

DJ['<EMA30>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=30)
DJ['<EMA50>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=50)
DJ['<EMA200>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=200)

def Upper(e):
    if e['<ISGREEN>'] : 
        return e['<HIGH>']-e['<CLOSE>']
    return e['<HIGH>']-e['<OPEN>']

def Lower(e):
    if e['<ISGREEN>'] : 
        return e['<OPEN>']-e['<LOW>']
    return e['<CLOSE>']-e['<LOW>']


DJ['<UPPER>'] = DJ.apply(Upper,axis=1)
DJ['<LOWER>'] = DJ.apply(Lower,axis=1)







In [ ]:

# Adding TA indicators : 

from ta.volatility import BollingerBands,KeltnerChannel,average_true_range
from ta.trend import MACD

DJ['<RSI>'] = ta.momentum.rsi(DJ['<CLOSE>'],window=15,fillna=1) / 100
# DJ['<CCI>'] = ta.trend.cci(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=14,fillna=1)


keltner = KeltnerChannel(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=50,window_atr=20,multiplier=3,fillna=1)
DJ['<KELTNER_H>'] = keltner.keltner_channel_hband()
DJ['<KELTNER_L>'] = keltner.keltner_channel_lband()
DJ['<KELTNER_M>'] = keltner.keltner_channel_mband()

DJ['<KELT_L_IND>'] = keltner.keltner_channel_lband_indicator()
DJ['<KELT_H_IND>'] = keltner.keltner_channel_hband_indicator()



bollinger =BollingerBands(close=DJ['<CLOSE>'],window=50,window_dev=3,fillna=True)
DJ['<BOL_H_IND>']=bollinger.bollinger_hband_indicator()
DJ['<BOL_L_IND>']=bollinger.bollinger_lband_indicator()

# macd = MACD(DJ['<CLOSE>'],fillna=True)


DJ['<ATR_24>'] = average_true_range(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=30).apply(lambda e : e/100)
DJ['<ATR_24_MULT>'] =abs(DJ['<SIZE>']) / DJ['<ATR_24>'] 


DJ["<ISGREEN>"] = DJ["<ISGREEN>"].astype(int)



In [ ]:
timeFrame = (int(DJ.iloc[1][1][1])-int(DJ.iloc[0][1][1]) ) * 60 + (int(DJ.iloc[1][1][3:5])-int(DJ.iloc[0][1][3:5]))
CandlesInDay = 24 * (60//timeFrame)
timeFrame
def candleToTime(j):
    minuteMult = CandlesInDay//24
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]

newnumparr = np.full((uniqueDaysCount*CandlesInDay,2),'',dtype=np.object_)


for i in range(uniqueDaysCount):
    for j in range(CandlesInDay):
        newnumparr[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j)]

newDF = pd.DataFrame(newnumparr,columns=['<DATE>','<TIME>'])

newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')


In [ ]:
newestDF.drop(range(CandlesInDay),inplace=True)
newestDF.fillna(0.0,inplace=True)

In [ ]:
startCandleHour = 1
startCandleIndex = int(startCandleHour * CandlesInDay )// 24

windowLengthHour = 22
windowLengthCount = int(windowLengthHour * CandlesInDay) // 24



# labelStartCandleHour = startCandleHour+windowLengthHour
# labelStartCandleIndex = int(labelStartCandleHour * CandlesInDay) // 24

labelWindowLengthHour = 2
labelWindowLengthCount = int(labelWindowLengthHour * CandlesInDay) // 24